# Retranscrire ses entretiens avec Whisper

[Whisper](https://github.com/openai/whisper) est le nom donné par Open IA (Chat GPT) à son système de reconnaissance vocale multilingue. Dans ce tutoriel, nous montrant deux façon d'utiliser Whisper:

1. en utilisant Whisper en "local", c'est-à-dire en installant Whisper sur son ordinateur.
2. en utilisant les services Huma-Num


## Whisper en local

Dans ce notebook, nous voyons comment utiliser Whisper -- l'outil de retranscription audio proposé par Open-ai -- en "local" à l'aide d'un script Python. C'est-à-dire qu'on installe whisper et le modèle sur sa machine.

Du fait de la popularité de Whisper, de nombreux tutoriaux qui fleurissent :

- https://www.css.cnrs.fr/whisper-pour-retranscrire-des-entretiens/
- https://analyzingalpha.com/openai-whisper-python-tutorial

###  Quel est l'intérêt ?

1. on maîtrise la "destinée" de ses données : elles restent privées. Il existe toutefois des applications web qui promettent aussi de protéger vos données comme [Whisper Web](https://whisper-web.pascal-mietlicki.fr/).

2. on apprend à charger un modèle. C'est aussi un prétexte pour découvrir la programmation avec Python et Jupyter pour les SHS.

3. Dans une utilisation (très) avancée, cela permet aussi de "fine-tuner" les modèles de Whisper afin qu'ils soient mieux adapter à ses propres corpus.

### les inconvénients

1. Pas toujours simple d'installer Whisper et les librairies nécessaires pour la retranscription (incompatibilité des versions, etc.). Ce n'est pas une solution "clé en main"

2. L'utilisation de Whisper demande beaucoup de ressources. Selon la performance de votre matériel informatique, la retranscription prendra plus ou moins de temps et vous ne pourrez pas nécessairement utiliser les modèles les plus "gros".


Pour éviter au maximum ces problèmes, nous utilisons le SSP Cloud de l'INSEE qui permet d'avoir le même espace de travail. 
Une autre solution, celle qu'on verra ensuite, est de faire appel aux services d'Huma-num lorsqu'on a des fichiers volumineux, de nombreux entretiens ou que l'on souhaite utiliser les modèles les plus performants.

### Installation des librairies Python

L'ensemble des librairies sont déjà installées. Toutefois si vous avez besoin de les réinstaller, il suffit de "décommenter" la ligne de code ci-dessous (i.e. enlever le '#') et de l'éxecuter.

#### Librairie pour manipuler des fichiers audios.

In [ ]:
#!pip install pydub audioop-lts

In [1]:
import pydub
from IPython.display import Audio, display #pour écouter les extraits audio avec un notebook jupyter

#### Installation du module Whisper et chargement des modèles

In [ ]:
import whisper
import time

tiny_model = whisper.load_model("tiny")
medium_model = whisper.load_model("medium")
large_model = whisper.load_model("large")

On peut déjà constater que la taille des modèles augmentent ainsi que le temps de chargement

### Charger et segmenter un mp3

On commence par charger l'entretien (au format mp3)

In [2]:
itw = pydub.AudioSegment.from_mp3("250618_rdv_ref_donnee_led.mp3") 

La ligne ci-dessous permet de prendre un extrait entre la seconde 30 et la seconde 90 (soit un extrait d'1 minute). La briéveté de l'extrait permettra de faire des tests avec plusieurs modèles.

In [3]:
extract = itw[30*1000:60*1000]

On enregistre notre extrait en exécutant la ligne ci-dessous. On remarque qu'un nouveau fichier sample.mp3 est apparu dans le gestionnaire de fichier à gauche

![](./figures/Capture_ecran_2025-12-08_15-33-24_sample.png)

In [4]:
pydub.AudioSegment.export(extract,"sample.mp3") #

<_io.BufferedRandom name='sample.mp3'>

### Pour écouter l'extrait audio

In [6]:
display(Audio('sample.mp3', autoplay=True))

## Retranscrire l'extrait

In [8]:
# Si jamais il y a une dépendance qui manque
#!pip install ffmpeg-python

Utiliser la fonction de transcription

In [9]:
start = time.time()
result_tiny = tiny_model.transcribe("sample.mp3", verbose=None, FP16=False)
print("Temps écoulé : ", time.ctime(time.time() - start)[11:19])


start = time.time()
result_medium = medium_model.transcribe("sample.mp3", FP16=False)
print("Temps écoulé : ", time.ctime(time.time() - start)[11:19])

start = time.time()
result_large = large_model.transcribe("sample.mp3", FP16=False)
print("Temps écoulé : ", time.ctime(time.time() - start)[11:19])


/opt/python/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Temps écoulé :  00:00:34


/opt/python/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Temps écoulé :  00:02:20


/opt/python/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Temps écoulé :  00:03:31


In [30]:

for result in [result_tiny, result_medium, result_large]:
    print("######### Résultat\n\n")
    for n, x in enumerate(result["segments"]):
        print(f"Segment {n} : {x["text"]}\n")

######### Résultat


Segment 0 :  C'est parce que les couleurs sont à l'adile et donc on va avoir trois déclamistes.

Segment 1 :  Ça donne des coups.

Segment 2 :  Ce que vous pouvez pas l'inclouer, ce que vous faites dans l'amour.

Segment 3 :  C'est un peu plus de plus de plus de plus de plus de plus de plus de plus de plus de plus.

Segment 4 :  On peut produire.

Segment 5 :  Ça va être forcément pas l'aide de nez, mais on a un temps de recherche de...

Segment 6 :  Ça va virer un cas de feu et co-agétion comme si des gens se sont touchants.

Segment 7 :  Ça va aller à terrorier, à terrorier, à...

Segment 8 :  Je pense que monبita est trit miteur le neuf.

######### Résultat


Segment 0 :  Et là c'est le D, le laboratoire d'économie, c'est ça le Découp ?

Segment 1 :  Est-ce que vous pouvez parler un peu de ce que vous faites dans le labo ?

Segment 2 :  En termes de données qu'on produit...

Segment 3 :  C'est un peu forcément parler de données, mais au moins en termes de recher

On constate que les résultats produits par le modèle "Tiny" sont très mauvais. Ils sont nettement meilleurs avec le modèle médium.

On constate également que la différence de résultats entre le modèle médium le modèle large n'est pas flagrante. Si on prend en compte la durée de la retranscription, on peut s'interroger sur l'intérêt d'utiliser le modèle large. 

Enfin, dans tous les cas, les modèles produisent des erreurs. Cela s'explique par le fait Whisper est un modèle génératif. Il va donc toujours proposé quelque chose même s'il n'a pas "compris". L'utilisation de Whisper ne dispense donc pas d'écouter les entretiens. Ainsi on va faciliter le travail de retranscription sans totalement perdre le lien avec le matériaux brut.

In [17]:
import pandas as pd
speech = pd.DataFrame.from_dict(result['segments'])
speech.head()

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0.0,5.4,"Et là c'est le D, le laboratoire d'économie, ...","[50364, 3790, 3684, 269, 6, 377, 476, 413, 11,...",0.0,-0.495057,1.62844,0.213604
1,1,0,5.4,8.4,Est-ce que vous pouvez parler un peu de ce qu...,"[50634, 4410, 12, 384, 631, 2630, 18248, 16421...",0.0,-0.495057,1.62844,0.213604
2,2,0,8.4,12.4,En termes de données qu'on produit...,"[50784, 2193, 1433, 279, 368, 40101, 421, 6, 2...",0.0,-0.495057,1.62844,0.213604
3,3,0,12.4,16.4,"C'est un peu forcément parler de données, mai...","[50984, 383, 6, 377, 517, 5604, 31358, 16421, ...",0.0,-0.495057,1.62844,0.213604
4,4,0,16.4,21.4,"Alors ça va varier, en face à l'éco-gestion, ...","[51184, 9946, 2788, 2773, 1374, 811, 11, 465, ...",0.0,-0.495057,1.62844,0.213604


## Mise en forme et sauvegarde

(inspiration Yacine Chitour) 

In [18]:
def convertir(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

In [19]:
convertir(100000)

'27:46:40'

In [20]:
# Chemin où vous souhaitez enregistrer votre transcription d'entretien en format .txt
entretien_transcrit = "entretien.txt"
 
# Enregistrement de la transcription
with open(entretien_transcrit , 'w', encoding='utf-8') as f:
    for segment in result["segments"]:
        start_time = convertir(segment['start'])
        end_time = convertir(segment['end'])
        f.write(f"{start_time} - {end_time}: {segment['text']}\n")

In [21]:
print(open("entretien.txt","r").read())

00:00:00 - 00:00:05:  Et là c'est le D, le laboratoire d'économie, c'est ça le Découp ?
00:00:05 - 00:00:08:  Est-ce que vous pouvez parler un peu de ce que vous faites dans le labo ?
00:00:08 - 00:00:12:  En termes de données qu'on produit...
00:00:12 - 00:00:16:  C'est un peu forcément parler de données, mais au moins en termes de recherche et de...
00:00:16 - 00:00:21:  Alors ça va varier, en face à l'éco-gestion, aussi des gens sont en co-gestion.
00:00:22 - 00:00:28:  Ça va aller de la théorique, on va dire, à du certainement statistique,
00:00:28 - 00:00:34:  les données sont individuelles, issues par exemple de décès ou apparentés,
00:00:34 - 00:00:37:  au niveau entreprise, etc.
00:00:37 - 00:00:40:  Donc ça, c'est plutôt des données qui préexistent.
00:00:40 - 00:00:47:  On va aller faire des données de marché social, des données de marché social, et enfin voilà.
00:00:47 - 00:00:59:  On va avoir des choses qui vont avoir un peu plus de tête...



Tester l'erreur 

- https://github.com/jitsi/jiwer